In [3]:
from pydantic import BaseModel, Field
import os, json
from dotenv import load_dotenv
from openai import OpenAI
from langsmith.wrappers import wrap_openai
from langsmith import traceable
from openai import pydantic_function_tool
load_dotenv()

# read in env variables
#api_key = os.getenv("OLLAMA_API_KEY")
#api_base = os.getenv("OLLAMA_API_BASE")
#model = os.getenv("QWEN3_API_MODEL")

api_key = os.getenv("GEMINI_API_KEY")
api_base = os.getenv("GEMINI_API_BASE")
model = os.getenv("GEMINI_API_MODEL")

In [4]:
model

'gemini-2.5-flash-lite'

In [5]:
# create a tool using pydantic BaseModel and Field, which reads a file and returns its contents
class ReadFileTool(BaseModel):
    """ Tool that reads a file and returns its contents. """
    name: str = Field(description="The name of the file to read.")
    def run(self) -> str:
        try:
            with open(self.name, 'r', encoding="utf-8") as file:
                return "####\n" + file.read() + "\n####"
        except Exception as e:
            return str(e)

class ReadFileList(BaseModel):
    """ Tool that reads files from a list of file names and returns the contents of all files in the list as a string. """
    textFiles: list = Field(description="The list files to read.")
    def run(self) -> str:
        merged_content = ""
        for file_path in self.textFiles:
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    merged_content += content + "\n"  # Add newline between files
            except FileNotFoundError:
                print(f"Warning: File '{file_path}' not found. Skipping.")
            except Exception as e:
                print(f"Error reading file '{file_path}': {e}")
        
        return merged_content.strip()  # Remove trailing newline
        
class FindTextFiles(BaseModel):
    """ Tool that finds text files in a directory and returns a list of text file names. """
    dirPath: str = Field(description="The path of the directory to read text files from")
    def run(self) -> list:
        """List various types of text files in the given directory"""
        text_extensions = {'.txt', '.text', '.log', '.md', '.rst', '.csv', '.json', '.xml'}
        text_files = []

        for file in os.listdir(self.dirPath):
            file_path = os.path.join(self.dirPath, file)
            if os.path.isfile(file_path):
            # Check file extension
                _, ext = os.path.splitext(file)
                if ext.lower() in text_extensions:
                    text_files.append(file)
        
        return text_files
    
class FindTextFilesAndMerge(BaseModel):
    """ Tool that finds text files in a directory and returns a string of their merged contents. """
    dirPath: str = Field(description="The path of the directory to read text files from")
    def run(self) -> list:
        """List various types of text files in the given directory"""
        text_extensions = {'.txt', '.text', '.log', '.md', '.rst', '.csv', '.json', '.xml'}
        text_files = []

        for file in os.listdir(self.dirPath):
            file_path = os.path.join(self.dirPath, file)
            if os.path.isfile(file_path):
            # Check file extension
                _, ext = os.path.splitext(file)
                if ext.lower() in text_extensions:
                    text_files.append(file)
            
        merged_content = ""
        for file_path in text_files:
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    merged_content += content + "\n"  # Add newline between files
            except FileNotFoundError:
                print(f"Warning: File '{file_path}' not found. Skipping.")
            except Exception as e:
                print(f"Error reading file '{file_path}': {e}")
        
        return merged_content.strip()  # Remove trailing newline

In [23]:
client = wrap_openai(OpenAI(
 api_key=api_key,
 base_url=api_base,
))

In [7]:
@traceable(name="Summarise Workflow")
def summarise_workflow():

 # message to read file
    messages = [
        {"role": "system", "content": "You are a useful assistant that reads files."},
        {"role": "user", "content": "Please open the file leinster.text and read its content."}
    ]

    response = client.chat.completions.create (
    model=model,
    messages=messages,
    temperature=0,
    tools=[pydantic_function_tool(ReadFileTool)]
    )

 # read the file
    tool_name = response.choices[0].message.tool_calls[0].function.name
    tool_args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
    file_content = ReadFileTool(**tool_args).run()

    messages = [
        {"role": "system", "content": "You are a useful assistant that summarises text."},
        {"role": "user", "content": "Please summarise the following text: {content}".format(content=file_content)}
    ]

    response = client.chat.completions.create (
    model=model,
    messages=messages,
    temperature=0,
    tools=[pydantic_function_tool(ReadFileTool)]
    )

    return response.choices[0].message.content

In [24]:
summarise_workflow()

"Leinster Rugby is one of Ireland's four professional rugby union teams, competing in the United Rugby Championship and European Rugby Champions Cup. They play their home games at the RDS Arena in Dublin, with larger matches held at Aviva Stadium. Founded in 1995, Leinster has been highly successful, winning nine domestic titles and four European Champions Cups. They are known for their blue jerseys and a crest featuring a harp."

In [9]:
# write function to execute tool. Take function name and parameters and return result of function
@traceable(name="AD Tool Call")
def execute_function(tool_call, tool_lookup):
    function_name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    tool = tool_lookup[function_name](**args)
    return function_name, tool.run()

In [ ]:
@traceable(name="react_loop")
def react_loop(messages, client, tools):
        tool_lookup = {tool.__name__: tool for tool in tools}
        tool_schemas = [pydantic_function_tool(tool) for tool in tools]
        while True:
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=0,
                tools=tool_schemas
            )
            # get the tool calls from the response
            tools_to_run = response.choices[0].message.tool_calls
            if not tools_to_run:
                #messages.append({"role": "assistant", "content": response.choices[0].message.content})
                break

            # execute the tool calls
            for tool_call in tools_to_run:
                function_name, tool_response = execute_function(tool_call, tool_lookup)
                print(f"executing {function_name}, \n tool response : \n {tool_response} ")
                # messages.append({"role": "assistant", "content": None, "function_call" : tool_call})
                #messages.append({"role": "function", "name": function_name, "content": tool_response})

            return response.choices[0].message.content

In [20]:
messages = [
    {"role": "system", "content": "You are a useful text files assistant that can locate text files in a given directory path, reads text files, merges their contents."},
    {
        "role": "user",
        "content": "Find text files in directory /home/aidodo/Foundations_of_AgenticAI/agentic-labs, then merge their contents of the returned list"
    }
]

In [19]:
tools = [FindTextFilesAndMerge]

In [22]:
response = react_loop(messages, client, tools)
print(response)

executing FindTextFilesAndMerge, 
 tool response : 
 Thomas Jeffrey Hanks (born July 9, 1956) is an American actor and filmmaker. Known for both his comedic and dramatic roles, he is one of the most popular and recognizable film stars worldwide, and is regarded as an American cultural icon.[2] Hanks is ranked as the fourth-highest-grossing American film actor.[3][4] His numerous awards include two Academy Awards, seven Emmy Awards, and four Golden Globe Awards; he has also been nominated for five BAFTA Awards and a Tony Award. He received the AFI Life Achievement Award in 2002, the Kennedy Center Honor in 2014, the Presidential Medal of Freedom in 2016, and the Golden Globe Cecil B. DeMille Award in 2020.[5][6][7]

Hanks rose to fame with leading roles in comedies Splash (1984), The Money Pit (1986), Big (1988), and A League of Their Own (1992). He won two consecutive Academy Awards for Best Actor, playing a gay lawyer suffering from AIDS in Philadelphia (1993), then the title characte

RateLimitError: Error code: 429 - [{'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15\nPlease retry in 59.782031507s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-2.5-flash-lite', 'location': 'global'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDel

In [16]:
@traceable(name="Summarise List Workflow")
def summarise_list_workflow():

 # message to read file
    messages = [
        {"role": "system", "content": "You are a useful text files assistant that can locate text files in a given directory path and Merge their contents"},
        {
            "role": "user",
            "content": "Find text files in directory /home/aidodo/Foundations_of_AgenticAI/agentic-labs and merge their contents."
        }
    ]

    response = client.chat.completions.create (
    model=model,
    messages=messages,
    temperature=0,
    tools=[pydantic_function_tool(FindTextFilesAndMerge)]
    )

 # read the file
    #tool_name = response.choices[0].message.tool_calls[0].function.name
    tool_args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
    merged_content = FindTextFilesAndMerge(**tool_args).run()
    #print(f"Printing File List \n##############\n {file_content}" )

    messages = [
        {"role": "system", "content": "You are a useful assistant that summarises text."},
        {"role": "user", "content": "Please summarise the following text: {content}".format(content=merged_content)}
    ]

    response = client.chat.completions.create (
        model=model,
        messages=messages,
        temperature=0,
        tools=[pydantic_function_tool(FindTextFilesAndMerge)]
    )

    return response.choices[0].message.content

In [17]:
summarise_list_workflow()

"The provided text contains information about two distinct entities: Tom Hanks and Leinster Rugby.\n\nTom Hanks is a highly acclaimed American actor and filmmaker, recognized globally as a cultural icon. He is known for his versatile roles in both comedy and drama, and is one of the highest-grossing actors in the US. Hanks has received numerous awards, including two Academy Awards, and has a notable history of collaborations with directors like Steven Spielberg, Ron Howard, Nora Ephron, and Robert Zemeckis.\n\nLeinster Rugby is one of Ireland's four professional rugby union teams, competing in the United Rugby Championship and the European Rugby Champions Cup. They primarily play at the RDS Arena in Dublin, with larger matches held at Aviva Stadium. Leinster has been a dominant force in Irish and European rugby since turning professional in 1995, having won multiple domestic titles and four European Champions Cups."